links:  
1. https://www.guardrailsai.com/docs/how_to_guides/using_llms#custom-llm-wrappers
2. https://freedium.cfd/https://medium.com/emalpha/safeguarding-llm-conversations-using-llama-guard-a1652da1d2de
3. https://github.com/guardrails-ai/profanity_free
4. https://www.guardrailsai.com/docs


## Objective
### We want to generate a vegan Mac-n-Cheese recipe as a list of ingredients and instructions. We will use Guardrails to make sure the recipe is vegan.

In [ ]:
! pip install guardrails-ai

In [33]:
import guardrails as gd
from rich import print

In [34]:
from guardrails.validators import Validator, register_validator, ValidationResult, PassResult, FailResult
from typing import Dict, Any

NON_VEGAN_INGREDIENTS = ["butter", "milk", "eggs", "cheese", "cream", "yogurt"]

SUBSTITUTIONS = {
    "butter": "margarine",
    "milk": "soy milk",
    "eggs": "flax eggs",
    "cheese": "vegan cheese",
    "cream": "soy cream",
    "yogurt": "soy yogurt",
}

@register_validator(name="is-vegan", data_type="string")
class IsVegan(Validator):

    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        global NON_VEGAN_INGREDIENTS, SUBSTITUTIONS

        # Make sure the ingredient is not in the list of non-vegan ingredients.
        if value.lower() in NON_VEGAN_INGREDIENTS:
            return FailResult(
                error_message=f"Value ${value} is not vegan.",
                # Programmatically fix the value by replacing it with a vegan
                # substitute.
                fix_value=SUBSTITUTIONS[value.lower()],
            )

        return PassResult()

## using RAIL structure 

In [18]:
rail_str = """
<rail version="0.1">

<output>
    <list name="ingredients" description="What are the ingredients for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="is-vegan" on-fail-is-vegan="fix" />
            <string name="brand" description="Suggested brand for the ingredient (if any)" />
            <bool name="optional" description="Is the ingredient necessary?" />
            <float name="quantity" format="units-imperial" />
            <string name="units" format="units-imperial" />
        </object>
    </list>
    <list name="instructions" description="What are the instructions for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="step" />
        </object>
    </list>
</output>
<prompt>
    Generate a recipe for vegan mac and cheese.
    ${gr.complete_xml_suffix}
</prompt>
<messages>
<message role="user">
Generate a recipe for vegan mac and cheese.
${gr.complete_xml_suffix}
</message>
</messages>

</rail>
"""

## using pydantic 


In [63]:
from pydantic import BaseModel, Field
from typing import List


prompt = """
Generate a recipe for vegan mac and cheese.
${gr.complete_xml_suffix}
"""

class Ingredient(BaseModel):
    index: int = Field(validators=[("1-indexed", "noop")]) #noop means no operation 
    name: str = Field(validators=[IsVegan(on_fail="fix")]) #call to action on fail is to Fix accordingly
    brand: str = Field(description="Suggested brand for the ingredient (if any)")
    optional: bool = Field(description="Is the ingredient necessary?")
    quantity: float = Field(description="how much of this ingredient to use", validators=[("units-imperial", "noop")])
    units: str = Field(validators=[("units-imperial", "noop")])

class Instruction(BaseModel):
    index: int = Field(validators=[("1-indexed", "noop")])
    step: str

class Recipe(BaseModel):
    ingredients: List[Ingredient] = Field(description="What are the ingredients for the recipe?")
    instructions: List[Instruction] = Field(description="What are the instructions for the recipe?")

## for RAIL structure 

In [35]:
guard = gd.Guard.for_rail_string(rail_str)

### For pydantic model

In [64]:
guard = gd.Guard.for_pydantic(output_class=Recipe)

In [65]:
raw_llm_response, validated_response, *rest = guard(
    messages=[{"role":"user", "content": prompt}],
    max_tokens=2048,
    temperature=0,
    model="gpt-4o-mini"
)

In [67]:
print(guard.history.first.iterations.first.inputs.messages[0]['content'])

Generate a recipe for mac and cheese.

Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
  <list description="What are the ingredients for the recipe?" name="ingredients" required="true">
    <object required="true">
      <integer name="index" required="true"></integer>
      <string format="is-vegan" name="name" required="true"></string>
      <string description="Suggested brand for the ingredient (if any)" name="brand" required="true"></string>
      <bool description="Is the ingredient necessary?" name="optional" required="true"></bool>
      <float description="how much of this ingredient to use" name="quantity" required="true"></float>
      <string name="units" required="true"></string>
    </object>
  </list>
  <list description="What are the instructions for the recipe?" name="instructions" required="true">
    <object required="true">
      <integer name="index" required="true"></integer>
      <string name="step" required="true"></string>
    </object>
  </list>
</output>

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [68]:
print(guard.history.last.tree)

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
│   │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
│   │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
│   │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
│   │ │ │ user │                                                                                              │ │ │
│   │ │ │      │ Generate a recipe for mac and cheese.                                                        │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ Given below is XML that describes the information to extract from this document and the tags │ │ │
│   │ │ │      │ to extract it into.                                                                          │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ <output>                                                                                     │ │ │
│   │ │ │      │   <list description="What are the ingredients for the recipe?" name="ingredients"            │ │ │
│   │ │ │      │ required="true">                                                                             │ │ │
│   │ │ │      │     <object required="true">                                                                 │ │ │
│   │ │ │      │       <integer name="index" required="true"></integer>                                       │ │ │
│   │ │ │      │       <string format="is-vegan" name="name" required="true"></string>                        │ │ │
│   │ │ │      │       <string description="Suggested brand for the ingredient (if any)" name="brand"         │ │ │
│   │ │ │      │ required="true"></string>                                                                    │ │ │
│   │ │ │      │       <bool description="Is the ingredient necessary?" name="optional"                       │ │ │
│   │ │ │      │ required="true"></bool>                                                                      │ │ │
│   │ │ │      │       <float description="how much of this ingredient to use" name="quantity"                │ │ │
│   │ │ │      │ required="true"></float>                                                                     │ │ │
│   │ │ │      │       <string name="units" required="true"></string>                                         │ │ │
│   │ │ │      │     </object>                                                                                │ │ │
│   │ │ │      │   </list>                                                                                    │ │ │
│   │ │ │      │   <list description="What are the instructions for the recipe?" name="instructions"          │ │ │
│   │ │ │      │ required="true">                                                                             │ │ │
│   │ │ │      │     <object required="true">                                                                 │ │ │
│   │ │ │      │       <integer name="index" required="true"></integer>                                       │ │ │
│   │ │ │      │       <string name="step" required="true"></string>                                          │ │ │
│   │ │ │      │     </object>                                                                                │ │ │
│   │ │ │      │   </list>                                                                                    │ │ │
│   │ │ │      │ </output>                                                                                    │ │ │
│   │ │ │      │                                   